# Array Basic

This tutorial will shows you basic operation on ndarray, such as:
- [printing](#Printing-an-Array)
- [element access](#Generic-Element-Access)
- [iterating an array](#Iterating-an-Array)
- [slicing](#Slicing-an-Array)

This notebook uses `xeus-cling` kernel to be able to use C++17 on notebook. For more info about `xeus-cling` please refer to https://github.com/jupyter-xeus/xeus-cling.

`nmtools` is an header-only library, to use it you only need to include necessary header. To add include path in `xeus-cling` simply add `pragma` for `cling`, for example:
```C++
#pragma cling add_include_path("/home/fahri/numeric_tools/include")
```

In [1]:
#pragma cling add_include_path("/home/fahri/numeric_tools/include")

In [2]:
#include "nmtools/array/ndarray.hpp"
#include "nmtools/array/view/slice.hpp"
#include "nmtools/array/index/ndenumerate.hpp"
#include "nmtools/utils/to_string.hpp"

#include <iostream>
#include <array>
#include <vector>

In [3]:
namespace nm = nmtools;
namespace na = nm::array;
using nm::utils::to_string;
using nm::index::ndindex;
using nm::view::slice;
using nm::shape;
using nm::None;
using std::tuple;

## Printing an Array

We can print a multidimensional array by simply converting the array to string and then use `std::cout`. `nmtools` provides utility function `nmtools::utils::to_string` to do such job, illustrated by the following examples:

In [4]:
{
    int a[2][3][2] = {
        {
            {1,2},
            {3,4},
            {5,6},
        },
        {
            { 7, 8},
            { 9,10},
            {11,12},
        },
    };
    std::cout << to_string(a) << std::endl;
    std::cout << "shape: " << to_string(shape(a)) << std::endl;
}

[[[	1	2],
[	3	4],
[	5	6]],

[[	7	8],
[	9	10],
[	11	12]]]
shape: [	2	3	2]


In [5]:
{
    auto a = na::fixed_ndarray{{
        {
            {1,2},
            {3,4},
            {5,6},
        },
        {
            { 7, 8},
            { 9,10},
            {11,12},
        },
    }};
    std::cout << to_string(a) << std::endl;
    std::cout << "shape: " << to_string(shape(a)) << std::endl;
}

[[[	1	2],
[	3	4],
[	5	6]],

[[	7	8],
[	9	10],
[	11	12]]]
shape: [	2	3	2]


In [6]:
{
    auto a = na::dynamic_ndarray({
        {
            {1,2},
            {3,4},
            {5,6},
        },
        {
            { 7, 8},
            { 9,10},
            {11,12},
        },
    });
    std::cout << to_string(a) << std::endl;
    std::cout << "shape: " << to_string(shape(a)) << std::endl;
}

[[[	1	2],
[	3	4],
[	5	6]],

[[	7	8],
[	9	10],
[	11	12]]]
shape: [	2	3	2]


## Generic Element Access

`nmtools` doesn't assume single ndarray type, hence we provide generic element accessor using `nmtools::at` that can be used in mutable and immutable functions.

In [7]:
{
    int a[1][2][1] = {{{1},{2}}};
    int b[2][2] = {{0,1},{2,3}};
    auto c = na::fixed_ndarray{{3,2,1}};
    auto d = na::dynamic_ndarray<int>();
    d.resize(1,1,2,1);
    d(0,0,1,0) = 1;
    std::cout << nm::at(a,0,0,0) << std::endl;
    std::cout << nm::at(b,0,1) << std::endl;
    std::cout << nm::at(c,2) << std::endl;
    std::cout << nm::at(d,0,0,1,0) << std::endl;
}

1
1
1
1


## Iterating an Array

`nmtools` doesn't limit the number of dimension for any ndarray. This can be problematic when we want to iterate the array without having to prepare nested loop. Instead of using nested loop, we can just iterate using single array using `nmtools::index::ndindex`, which takes the shape of an array and generate "indices pack" for corresponding array to be used in single loop. Then to read element of an array given packed indices, we can use `nmtools::apply_at`.

One can also use `nmtools::index::ndenumerate` to get generated index with its corresponding elements.

In [8]:
{
    auto a = na::dynamic_ndarray<float>();
    a.resize(2,3,2);
    std::cout << "before:\n" << to_string(a) << std::endl;
    
    auto shape_ = a.shape();
    auto indices = ndindex(shape_);
    // just fill with index
    for (size_t i=0; i<indices.size(); i++) {
        auto idx = indices[i];
        nm::apply_at(a,idx) = i + 0.1*(i+1);
    }
    std::cout << "after:\n" << to_string(a) << std::endl;
}

before:
[[[	0.000000	0.000000],
[	0.000000	0.000000],
[	0.000000	0.000000]],

[[	0.000000	0.000000],
[	0.000000	0.000000],
[	0.000000	0.000000]]]
after:
[[[	0.100000	1.200000],
[	2.300000	3.400000],
[	4.500000	5.600000]],

[[	6.700000	7.800000],
[	8.900000	10.000000],
[	11.100000	12.200000]]]


In [9]:
{
    double a[1][1][2][3][1][1][2] = {};
    std::cout << "before:\n" << to_string(a) << std::endl;
    
    auto indices = ndindex(shape(a));
    // just fill with index
    for (size_t i=0; i<indices.size(); i++) {
        auto idx = indices[i];
        nm::apply_at(a,idx) = i + 0.1*(i+1);
    }
    std::cout << "after:\n" << to_string(a) << std::endl;
}

before:
[[[[[[[	0.000000	0.000000]]],


[[[	0.000000	0.000000]]],


[[[	0.000000	0.000000]]]],



[[[[	0.000000	0.000000]]],


[[[	0.000000	0.000000]]],


[[[	0.000000	0.000000]]]]]]]
after:
[[[[[[[	0.100000	1.200000]]],


[[[	2.300000	3.400000]]],


[[[	4.500000	5.600000]]]],



[[[[	6.700000	7.800000]]],


[[[	8.900000	10.000000]]],


[[[	11.100000	12.200000]]]]]]]


In [10]:
{
    using nm::index::ndenumerate;
    auto a = na::dynamic_ndarray<float>();
    a.resize(2,3,2);
    // assignment
    a = {
        {
            {0,1},
            {2,3},
            {4,5},
        },
        {
            { 6, 7},
            { 8, 9},
            {10,11},
        }
    };
    
    auto pack = ndenumerate(a);
    std::cout << "index\t\t\t     value" << std::endl;
    for (size_t i=0; i<pack.size(); i++) {
        auto [index, value] = pack[i];
        std::cout << to_string(index) << " : " << to_string(value) << std::endl;
    }

    // also note that "pack" takes reference to a, changes to a reflected to pack
    a(0,2,0) = 77;

    std::cout << "using range for:" << std::endl;
    for (auto [index, value] : pack)
        std::cout << to_string(index) << " : " << to_string(value) << std::endl;
}

index			     value
[	0	0	0] : 0.000000
[	0	0	1] : 1.000000
[	0	1	0] : 2.000000
[	0	1	1] : 3.000000
[	0	2	0] : 4.000000
[	0	2	1] : 5.000000
[	1	0	0] : 6.000000
[	1	0	1] : 7.000000
[	1	1	0] : 8.000000
[	1	1	1] : 9.000000
[	1	2	0] : 10.000000
[	1	2	1] : 11.000000
using range for:
[	0	0	0] : 0.000000
[	0	0	1] : 1.000000
[	0	1	0] : 2.000000
[	0	1	1] : 3.000000
[	0	2	0] : 77.000000
[	0	2	1] : 5.000000
[	1	0	0] : 6.000000
[	1	0	1] : 7.000000
[	1	1	0] : 8.000000
[	1	1	1] : 9.000000
[	1	2	0] : 10.000000
[	1	2	1] : 11.000000


### Slicing an Array

Other basic operation on array is slicing. `nmtools` provides python/numpy-inspired slicing function `slice`. The function accept a source array and variadic slice parameter. There is also special constant `nmtools::None` that simulates python's `None` to achieve similar slicing effects.

The following examples show you how to use start, stop, and step for each axis, using `None` constant, as well as mixing indexing with slicing. Note that currently full indexing using slice is not allowed and slice parameter should take at least start and stop values.

```Python
# assume a is 2-dimensional
sliced = a[:,:]
# also note that the expression is equivalent to:
sliced = a[None:None,None:None]
```
is equal to
```C++
auto slice0 = tuple{None,None};
auto slice1 = tuple{None,None};
auto sliced = slice(a,slice0,slice1};
```

In [11]:
{
    int a[2][3] = {
        {0,1,2},
        {3,4,5},
    };
    auto slice0 = tuple{/*start=*/None,/*stop=*/None};
    auto slice1 = tuple{/*start=*/None,/*stop=*/None,/*step=*/-1};
    auto sliced = slice(a,slice0,slice1);
    std::cout << "original:\n" << to_string(a) << std::endl;
    std::cout << "sliced:\n" << to_string(sliced) << std::endl;
}

original:
[[	0	1	2],
[	3	4	5]]
sliced:
[[	2	1	0],
[	5	4	3]]


In [12]:
{
    int array[2][3][2] = {
        {
            {0,1},
            {2,3},
            {4,5},
        },
        {
            {6,7},
            {8,9},
            {10,11},
        }
    };
    auto slice0 = tuple{1,None,-1};
    auto slice1 = tuple{None,-1};
    auto slice2 = tuple{None,None,-1};
    auto sliced = slice(array,slice0,slice1,slice2);
    std::cout << "original:\n" << to_string(array) << std::endl;
    std::cout << "sliced:\n" << to_string(sliced) << std::endl;
}

original:
[[[	0	1],
[	2	3],
[	4	5]],

[[	6	7],
[	8	9],
[	10	11]]]
sliced:
[[[	7	6],
[	9	8]],

[[	1	0],
[	3	2]]]


In [13]:
{
    int array[2][3][2] = {
        {
            {0,1},
            {2,3},
            {4,5},
        },
        {
            {6,7},
            {8,9},
            {10,11},
        }
    };
    auto slice0 = tuple{1,None,-1};
    auto slice1 = 1;
    auto slice2 = tuple{None,None,-1};
    auto sliced = slice(array,slice0,slice1,slice2);
    std::cout << "original:\n" << to_string(array) << std::endl;
    std::cout << "sliced:\n" << to_string(sliced) << std::endl;
}

original:
[[[	0	1],
[	2	3],
[	4	5]],

[[	6	7],
[	8	9],
[	10	11]]]
sliced:
[[	9	8],
[	3	2]]
